<a href="https://colab.research.google.com/github/diana-07/Diana_tests_DL/blob/main/project_DL_Diana_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# #eliminar pasta dataset caso seja necessário
# import shutil
# import os

# os.chdir('/content')
# print(os.getcwd())

# # Caminho para a pasta a apagar
# dir_path = "dataset"

# # Só apaga se existir
# if os.path.exists(dir_path):
#     shutil.rmtree(dir_path)
#     print(f"Pasta '{dir_path}' apagada com sucesso.")
# else:
#     print(f"Pasta '{dir_path}' não existe.")


In [4]:
import os
import shutil
import sys

#IMPORTANTE: BAIXAR ANTES O FICHEIRO prepare_data.py

# 1. Remover pasta 'dataset' se já existir
if os.path.exists("dataset"):
    shutil.rmtree("dataset")
    print("Pasta 'dataset' removida.")

# 2. Clonar o repositório com checkout parcial
os.system("git clone --filter=blob:none --no-checkout https://github.com/THETIS-dataset/dataset.git")

# 3. Entrar na pasta 'dataset' e configurar checkout parcial
os.chdir("dataset")
os.system("git sparse-checkout init --cone")
os.system("git sparse-checkout set VIDEO_Skelet3D")
os.system("git checkout")
print("Clone e checkout parcial concluídos.")

# 4. Voltar ao diretório anterior
os.chdir("..")


# 5. Procurar prepare_data.py fora da pasta 'dataset'
nome_arquivo = "prepare_data.py"
caminho_encontrado = None

for raiz, dirs, arquivos in os.walk("."):
    if nome_arquivo in arquivos and "dataset" not in raiz:
        caminho_encontrado = os.path.join(raiz, nome_arquivo)
        break

# 6. Mover o prepare_data.py para dentro de 'dataset'
if caminho_encontrado:
    destino_final = os.path.join("dataset", nome_arquivo)
    shutil.move(caminho_encontrado, destino_final)
    print(f"{nome_arquivo} movido para a pasta 'dataset'.")
else:
    print(f"Arquivo {nome_arquivo} não encontrado fora da pasta 'dataset'.")

# 7. Adicionar 'dataset' ao sys.path para permitir importações
dataset_path = os.path.abspath("dataset")
if dataset_path not in sys.path:
    sys.path.append(dataset_path)

# 8. Agora já podes importar a função
try:
    from prepare_data import collect_by_players
    from prepare_data import pad_or_truncate_keypoints
    from prepare_data import extract_player_id
    print("Importação bem-sucedida.")
except ImportError as e:
    print("Erro ao importar função:", e)






Pasta 'dataset' removida.
Clone e checkout parcial concluídos.
prepare_data.py movido para a pasta 'dataset'.
Importação bem-sucedida: collect_by_players disponível.


In [5]:
import os
import cv2
import torch
import numpy as np
from tqdm import tqdm
from collections import defaultdict
from sklearn.model_selection import train_test_split
import random
import pickle

# from prepare_data import pad_or_truncate_keypoints
# from prepare_data import extract_player_id


In [6]:
!pip uninstall -y mediapipe-silicon
!pip uninstall -y mediapipe
!pip uninstall -y protobuf
!pip install protobuf==3.20.3  # Specific version that often works well with MediaPipe
!pip install mediapipe

Found existing installation: mediapipe 0.10.21
Uninstalling mediapipe-0.10.21:
  Successfully uninstalled mediapipe-0.10.21
Found existing installation: protobuf 4.25.7
Uninstalling protobuf-4.25.7:
  Successfully uninstalled protobuf-4.25.7
  Using cached protobuf-3.20.3-py2.py3-none-any.whl.metadata (720 bytes)
Using cached protobuf-3.20.3-py2.py3-none-any.whl (162 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.17.1 requires protobuf<6.0.0,>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.
ydf 0.11.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incompatible.


  Using cached mediapipe-0.10.21-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (9.7 kB)
  Using cached protobuf-4.25.7-cp37-abi3-manylinux2014_x86_64.whl.metadata (541 bytes)
Using cached mediapipe-0.10.21-cp311-cp311-manylinux_2_28_x86_64.whl (35.6 MB)
Using cached protobuf-4.25.7-cp37-abi3-manylinux2014_x86_64.whl (294 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.7 which is incompatible.
ydf 0.11.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 4.25.7 which is incompatible.


In [7]:
import mediapipe as mp


In [9]:
%cd dataset
!python prepare_data.py

/content/dataset
CUDA available: True
Device name: Tesla T4
['backhand_slice', 'forehand_openstands', 'forehand_flat', 'kick_service', 'flat_service', 'backhand_volley', 'backhand', 'smash', 'slice_service', 'forehand_slice', 'forehand_volley', 'backhand2hands']
Fim da execução do prepare_data.py


In [10]:

#extract the keypoints AND pad/truncate

root_dir = "npy_videos"

mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False) # This allows MediaPipe to track pose across frames, potentially leading to more stable and accurate keypoint estimations.

all_keypoints = {}

# Top-level loop with a single progress bar
action_folders = sorted([d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))])
for action_folder in tqdm(action_folders, desc="Processing actions"):
    action_path = os.path.join(root_dir, action_folder)
    all_keypoints[action_folder] = {}

    video_files = sorted([f for f in os.listdir(action_path) if f.endswith(".npy")])
    for video_file in video_files:  # No tqdm here
        video_path = os.path.join(action_path, video_file)
        try:
            sample = np.load(video_path)
            video_keypoints = []

            for frame in sample:
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = pose.process(frame_rgb)

                if results.pose_landmarks:
                    keypoints = [[lm.x, lm.y, lm.z] for lm in results.pose_landmarks.landmark]
                else:
                    keypoints = np.zeros((33, 3)).tolist()

                video_keypoints.append(keypoints)

            video_keypoints = np.array(video_keypoints)
            video_keypoints = pad_or_truncate_keypoints(video_keypoints, target_length=120)
            all_keypoints[action_folder][video_file] = video_keypoints





        except Exception as e:
            pass  # Suppress per-video error printing for a clean log

# Final summary
print("\n Summary of extracted keypoints:")
for action, videos in all_keypoints.items():
    print(f"- {action}: {len(videos)} videos processed.")

# Jsut to see the calues for the first one
first_action = list(all_keypoints.keys())[0]
first_video = list(all_keypoints[first_action].keys())[0]
print(f"Keypoints for the first frame of {first_video} in {first_action}:")
print(all_keypoints[first_action][first_video][0])


Processing actions: 100%|██████████| 12/12 [23:00<00:00, 115.04s/it]


 Summary of extracted keypoints:
- backhand: 97 videos processed.
- backhand2hands: 107 videos processed.
- backhand_slice: 100 videos processed.
- backhand_volley: 103 videos processed.
- flat_service: 96 videos processed.
- forehand_flat: 110 videos processed.
- forehand_openstands: 101 videos processed.
- forehand_slice: 97 videos processed.
- forehand_volley: 93 videos processed.
- kick_service: 109 videos processed.
- slice_service: 100 videos processed.
- smash: 104 videos processed.
Keypoints for the first frame of p10_backhand_skelet3D_s2.npy in backhand:
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Train: 850, Val: 177, Test: 190


In [ ]:
#Organize videos by player and split player IDs

# Organize videos by player
player_video_map = defaultdict(list)
for action, videos in all_keypoints.items():
    for video_file in videos:
        player_id = extract_player_id(video_file)
        player_video_map[player_id].append((action, video_file))

# Split player IDs
players = list(player_video_map.keys())
random.shuffle(players)

train_end = int(len(players) * 0.7)
val_end = int(len(players) * 0.85)

train_players = players[:train_end]
val_players = players[train_end:val_end]
test_players = players[val_end:]

train_set = collect_by_players(train_players, player_video_map, all_keypoints)
val_set = collect_by_players(val_players,  player_video_map, all_keypoints)
test_set = collect_by_players(test_players,  player_video_map, all_keypoints)

print(f"Train: {len(train_set)}, Val: {len(val_set)}, Test: {len(test_set)}")